В стандартной установке может отсутствовать часть модулей, необходимых для работы, их нужно установить  
Команда установки "pip install имя_модуля" вводится в этом же окне, но в формате кода.  
После установки необходимо перезапустить ядро выбрав в меню сверху Kernel->Restart.

In [ ]:
import pandas as pd  # Для работы с таблицами данных (ДатаФреймы)
import requests  # Для запросов к серверу ИСС
import json  # Для обработки ответов сервера ИСС
from datetime import datetime  # Для работы с датами
import time  # Для работы со временем (пауза между запросами)
import numpy as np  # Для работы с одномерными массивами

# Выгрузка данных по акции Сбербанка
Сперва зададим изначально необходимые значения  
Сформируем базовый заспрос и пустой дата фрейм с именами колонок

In [ ]:
# В базовом запросе мы укажем:
# engines/stok (фондовый рынок), markets/shares - рынок акций, boards/TQBR - режим торгов для акции сбербанка TQBR
# Базовый запрос поместим в переменную base_url
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url)  # Получим ответ от сервера
result = json.loads(response.text)  # Расшифровываем ответ от сервера
# Задаем имена колонок извлекая данные из ответа сервера
col_name = result['history']['columns']
data_shares = pd.DataFrame(columns=col_name)

In [ ]:
data_shares  # Можно посмотреть что получилось

Получился датафрйем из 23 колонок  
Перейдем к получению данных по акциям Сбербанка

In [ ]:
# Первая страница данных
url_share = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/SBER.json'
# Получаем данные
response = requests.get(url_share)
result = json.loads(response.text)
resp_date = result['history']['data']
# Извлеченные данные добавляем в начало дата фрейма
data_shares = pd.DataFrame(resp_date, columns=col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

Если же количество строк 100, то получены не все данные (сервер отправляет ограниченное количество строк за раз).  
В этом случае запускаем в цикле переборку остальных страниц.  
Цикл будет работать до тех пор, пока мы не получим количество строк меньше, чем 100.  
Для этого каждый раз на предпоследнем шаге цикла переопределяем переменную.

In [ ]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page = url_share + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns=col_name)
    data_shares = pd.concat([data_shares, data_next_page], ignore_index=True)
    a = len(resp_date)
    b = b + 100

Посмотрим сколько примерно строк получилось, узнав значение b

In [ ]:
b

За короткое время было загружено более 2600 строк данных  
Точное колисество строк можно узнать отобразил наш датафрейм

In [ ]:
data_shares  # Посмотрим, что получилось

In [ ]:
data_shares.to_csv('data_shares', sep=';', encoding = 'cp1251') # Сохраним данные в csv файл

## Данные по фьючерсам  

In [ ]:
# Получим данные по фьючерсам за текущий день
base_url = 'https://iss.moex.com/iss/history/engines/futures/markets/forts/boards/RFUD/securities.json'
date_start = datetime.strptime(
    "2023-01-01", "%Y-%m-%d")  # Указываем начальную дату

In [ ]:
# Получим торговые дни
# Ссылка на календарь праздников
nwd_url = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/calendar.json'
nwd_response = requests.get(nwd_url)
nwd_result = json.loads(nwd_response.text)
no_work_day = pd.DataFrame(nwd_result['holidays'], columns=['holidays'])
no_work_day['holidays_date'] = pd.to_datetime(
    no_work_day['holidays'], format="%Y-%m-%d")
no_work_day = no_work_day[no_work_day['holidays_date'] >= date_start]
datelist = pd.date_range(start="2023-01-01", end="2023-09-29")
datelist = pd.DataFrame(list(datelist.to_pydatetime()),
                        columns=['holidays_date'])
day_list = datelist["holidays_date"].values  # Список дат в диапазоне
nwd_list = no_work_day["holidays_date"].values  # Список праздничныйх дней
traide_day = []
for i in day_list:
    if i not in nwd_list:
        traide_day.append(i)

In [ ]:
response = requests.get(base_url)  # Базовый адрес
result = json.loads(response.text)
col_name = result['history']['columns']  # Задаем имена колонок
data_f = pd.DataFrame(columns=col_name)
# В цикле по торговым дням получим  данные с МосБиржи по фьючерсам
for i in traide_day:
    # Передаем дату в виде строки формата ГГГГ-ММ-ДД
    url_date = np.datetime_as_string(i, unit='D')
    url_one_page = base_url + '?date=' + url_date
    response = requests.get(url_one_page)  # Базовый адрес и дата
    col_name = result['history']['columns']  # Задаем имена колонок
    result = json.loads(response.text)
    # Узнаем количество строк, если их 0 (нет торгов), то берем следующий день
    total_line = result['history.cursor']['data'][0][1]
    if total_line == 0:
        continue
    step = total_line//100  # Узнаем количество сотен
    ost = total_line % 100  # Узнаем количество едениц
    # Извлекаем полученные на 1 странице данные
    resp_date = result['history']['data']
    # Полеченные данные записываем в датафрейм
    data_p_one = pd.DataFrame(resp_date, columns=col_name)
    # Записываем в изначальный датафрейм полученные данные
    data = pd.concat([data_f, data_p_one], ignore_index=True)
    # Цикл загрузки остальных строк
    for j in range(1, step+1):
        page_line = '&start=' + str(j*100)
        url_next_page = url_one_page + page_line
        response = requests.get(url_next_page)
        result = json.loads(response.text)
        resp_date = result['history']['data']
        data_next_page = pd.DataFrame(resp_date, columns=col_name)
        data_f = pd.concat([data_f, data_next_page], ignore_index=True)
        time.sleep(1.5)

In [ ]:
data_f

## Данные по валютным торгам

In [ ]:
base_url = 'http://iss.moex.com/iss/history/engines/currency/markets/selt/securities.json'

In [ ]:
response = requests.get(base_url)  # Базовый адрес
result = json.loads(response.text)
col_name = result['history']['columns']  # Задаем имена колонок
data_cur = pd.DataFrame(columns=col_name)
# В цикле по торговым дням получим  данные с МосБиржи по фьючерсам
for i in traide_day:
    # Передаем дату в виде строки формата ГГГГ-ММ-ДД
    url_date = np.datetime_as_string(i, unit='D')
    url_one_page = base_url + '?date=' + url_date
    response = requests.get(url_one_page)  # Базовый адрес и дата
    col_name = result['history']['columns']  # Задаем имена колонок
    result = json.loads(response.text)
    # Узнаем количество строк, если их 0 (нет торгов), то берем следующий день
    total_line = result['history.cursor']['data'][0][1]
    if total_line == 0:
        continue
    step = total_line//100  # Узнаем количество сотен
    ost = total_line % 100  # Узнаем количество едениц
    # Извлекаем полученные на 1 странице данные
    resp_date = result['history']['data']
    # Полеченные данные записываем в датафрейм
    data_p_one = pd.DataFrame(resp_date, columns=col_name)
    # Записываем в изначальный датафрейм полученные данные
    data = pd.concat([data_cur, data_p_one], ignore_index=True)
    # Цикл загрузки остальных строк
    for j in range(1, step+1):
        page_line = '&start=' + str(j*100)
        url_next_page = url_one_page + page_line
        response = requests.get(url_next_page)
        result = json.loads(response.text)
        resp_date = result['history']['data']
        data_next_page = pd.DataFrame(resp_date, columns=col_name)
        data_cur = pd.concat([data_cur, data_next_page], ignore_index=True)
        time.sleep(1.5)